# 4.1.2 特徵描述子與特徵匹配 (Feature Descriptors & Matching)

**WBS 4.1.2**: 特徵描述子、特徵匹配與影像配準

## 學習目標
- 理解特徵描述子的原理與應用
- 掌握 SIFT, SURF, ORB, BRIEF, BRISK 特徵檢測器
- 學習特徵匹配算法 (BFMatcher, FLANN)
- 應用 Homography 進行影像配準
- 比較不同特徵檢測器的性能

**難度等級**: ⭐⭐⭐⭐ (進階)  
**預估時間**: 120 分鐘  
**WBS編號**: 4.1.2

## 環境設置與導入

In [ ]:
import sys
import os

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import cv2
import numpy as np
import matplotlib.pyplot as plt
from utils.image_utils import load_image, resize_image
from utils.visualization import display_image, display_multiple_images
from utils.performance import time_function, benchmark_function

print(f'✅ OpenCV version: {cv2.__version__}')
print(f'✅ NumPy version: {np.__version__}')

# Check for xfeatures2d (for SIFT/SURF)
try:
    cv2.xfeatures2d.SIFT_create()
    print('✅ SIFT available (cv2.xfeatures2d)')
except AttributeError:
    try:
        cv2.SIFT_create()
        print('✅ SIFT available (cv2.SIFT_create)')
    except:
        print('⚠️ SIFT not available')

## 1. 特徵描述子概述

### 什麼是特徵描述子？

**特徵描述子 (Feature Descriptor)** 是用向量來描述影像中關鍵點周圍區域的特徵，使得這些特徵點可以被唯一識別和匹配。

### 核心概念

```
特徵檢測與描述的流程:

1. 特徵檢測 (Feature Detection)
   └─> 找到影像中的關鍵點 (Keypoints)
       - 角點、斑點、邊緣等

2. 特徵描述 (Feature Description)
   └─> 為每個關鍵點建立描述向量
       - SIFT: 128維向量
       - ORB: 256位元二進位字串

3. 特徵匹配 (Feature Matching)
   └─> 在不同影像間尋找相同的特徵點
       - 比較描述子的相似度
```

### 理想特徵描述子的特性

| 特性 | 說明 | 重要性 |
|------|------|--------|
| **尺度不變性** | Scale Invariance | 影像縮放時特徵不變 |
| **旋轉不變性** | Rotation Invariance | 影像旋轉時特徵不變 |
| **光照不變性** | Illumination Invariance | 亮度變化時特徵穩定 |
| **仿射不變性** | Affine Invariance | 視角變化時特徵穩定 |
| **區分性** | Distinctiveness | 不同特徵有明顯差異 |
| **計算效率** | Computational Efficiency | 運算速度快 |

## 2. SIFT (Scale-Invariant Feature Transform)

### 2.1 SIFT 原理

**SIFT** 是最經典的特徵檢測器，由 David Lowe 於 1999 年提出。

#### 核心思想

```
SIFT 演算法步驟:

1. 尺度空間極值檢測
   └─> 使用 DoG (Difference of Gaussian) 找特徵點

2. 關鍵點定位
   └─> 精確定位關鍵點位置，去除不穩定點

3. 方向分配
   └─> 為每個關鍵點分配主方向（旋轉不變性）

4. 關鍵點描述
   └─> 在關鍵點周圍 16x16 區域計算梯度直方圖
       生成 128 維特徵向量 (4x4x8)
```

#### SIFT 特點

- ✅ **尺度不變**: 可檢測不同尺寸的特徵
- ✅ **旋轉不變**: 影像旋轉不影響檢測
- ✅ **穩定性高**: 對光照、噪聲有強魯棒性
- ❌ **計算緩慢**: 運算量大，不適合實時應用
- ⚠️ **專利限制**: 商業使用需授權（2020年已過期）

#### SIFT 數學原理

**DoG (Difference of Gaussian)**:
```
DoG(x,y,σ) = G(x,y,kσ) - G(x,y,σ)
```

其中 G 是高斯濾波器，k 是尺度因子。

In [ ]:
# Load test image
img_path = '../assets/images/basic/opencv.jpg'
if os.path.exists(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (600, 400))
else:
    # Create synthetic test image
    img = np.ones((400, 600, 3), dtype=np.uint8) * 255
    cv2.rectangle(img, (100, 100), (500, 300), (0, 0, 0), -1)
    cv2.circle(img, (300, 200), 80, (255, 255, 255), -1)
    
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Create SIFT detector
try:
    sift = cv2.xfeatures2d.SIFT_create()
except AttributeError:
    sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints, descriptors = sift.detectAndCompute(gray, None)

# Draw keypoints
img_sift = cv2.drawKeypoints(img, keypoints, None, 
                              flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Display
display_multiple_images(
    [img, img_sift],
    ['Original Image', f'SIFT Keypoints ({len(keypoints)} detected)'],
    rows=1, cols=2, figsize=(15, 5)
)

print(f'\n✅ SIFT 檢測結果:')
print('='*60)
print(f'關鍵點數量: {len(keypoints)}')
print(f'描述子維度: {descriptors.shape if descriptors is not None else "None"}')
print(f'描述子類型: {descriptors.dtype if descriptors is not None else "None"}')

if len(keypoints) > 0:
    kp = keypoints[0]
    print(f'\n第一個關鍵點資訊:')
    print(f'  位置: ({kp.pt[0]:.1f}, {kp.pt[1]:.1f})')
    print(f'  尺度: {kp.size:.2f}')
    print(f'  角度: {kp.angle:.2f}°')
    print(f'  響應: {kp.response:.4f}')

### 2.2 SIFT 尺度不變性驗證

In [ ]:
# Test SIFT scale invariance
scales = [0.5, 1.0, 1.5, 2.0]
results = []
titles = []

for scale in scales:
    # Resize image
    h, w = gray.shape
    scaled = cv2.resize(gray, (int(w*scale), int(h*scale)))
    
    # Detect keypoints
    kp, desc = sift.detectAndCompute(scaled, None)
    
    # Draw keypoints
    img_kp = cv2.cvtColor(scaled, cv2.COLOR_GRAY2BGR)
    img_kp = cv2.drawKeypoints(img_kp, kp, None, color=(0, 255, 0))
    
    results.append(img_kp)
    titles.append(f'Scale {scale}x\n{len(kp)} keypoints')

display_multiple_images(results, titles, rows=1, cols=4, figsize=(20, 5))

print('\n💡 觀察: SIFT 在不同尺度下都能檢測到穩定的特徵點')

## 3. SURF (Speeded-Up Robust Features)

### 3.1 SURF 原理

**SURF** 是 SIFT 的加速版本，由 Bay et al. 於 2006 年提出。

#### 核心改進

```
SURF vs SIFT:

1. 積分影像 (Integral Image)
   └─> 快速計算矩形區域的和

2. Hessian 矩陣近似
   └─> 使用盒子濾波器代替高斯濾波器

3. 64維描述子
   └─> 比 SIFT 的 128 維更快

速度: SURF ≈ 3-7x 快於 SIFT
```

#### SURF 特點

- ✅ **速度快**: 比 SIFT 快 3-7 倍
- ✅ **魯棒性好**: 對模糊和旋轉有良好的抗性
- ❌ **專利保護**: 需要授權才能商業使用
- ⚠️ **OpenCV 限制**: 某些版本需要 contrib 模組

### ⚠️ 注意: SURF 演算法已申請專利

在某些 OpenCV 配置中，SURF 可能不可用。

In [ ]:
# Try to use SURF (may not be available)
try:
    surf = cv2.xfeatures2d.SURF_create()
    
    # Detect keypoints
    keypoints_surf, descriptors_surf = surf.detectAndCompute(gray, None)
    
    # Draw keypoints
    img_surf = cv2.drawKeypoints(img, keypoints_surf, None, 
                                  flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    display_multiple_images(
        [img, img_surf],
        ['Original', f'SURF Keypoints ({len(keypoints_surf)} detected)'],
        rows=1, cols=2, figsize=(15, 5)
    )
    
    print(f'✅ SURF 檢測結果:')
    print(f'關鍵點數量: {len(keypoints_surf)}')
    print(f'描述子維度: {descriptors_surf.shape}')
    
except (AttributeError, cv2.error) as e:
    print('⚠️ SURF 在此配置中不可用')
    print('   原因: SURF 演算法已申請專利，某些 OpenCV 版本將其排除')
    print('   替代方案: 使用 ORB 或 BRISK（開源且速度快）')

## 4. ORB (Oriented FAST and Rotated BRIEF)

### 4.1 ORB 原理

**ORB** 是由 OpenCV 實驗室開發的開源替代方案，結合了 FAST 關鍵點檢測器和 BRIEF 描述子。

#### 核心特性

```
ORB = FAST + BRIEF + 改進

1. FAST 角點檢測
   └─> 快速檢測關鍵點

2. Harris 角點度量
   └─> 選擇最好的關鍵點

3. 方向分配
   └─> 計算 intensity centroid（旋轉不變）

4. rBRIEF 描述子
   └─> 旋轉感知的 BRIEF（256 位元二進位）
```

#### ORB 優勢

- ✅ **完全開源**: 無專利限制
- ✅ **速度極快**: 比 SIFT 快 100 倍以上
- ✅ **旋轉不變**: 對旋轉有很好的魯棒性
- ✅ **二進位描述子**: 匹配速度快（漢明距離）
- ⚠️ **尺度不變性弱**: 相比 SIFT 較弱

#### 性能比較

```
速度排名: ORB > SURF > SIFT
魯棒性:   SURF ≥ SIFT > ORB
適用場景: ORB 最適合實時應用
```

In [ ]:
# Create ORB detector
orb = cv2.ORB_create(nfeatures=1000)  # Max 1000 keypoints

# Detect keypoints and compute descriptors
keypoints_orb, descriptors_orb = orb.detectAndCompute(gray, None)

# Draw keypoints
img_orb = cv2.drawKeypoints(img, keypoints_orb, None, color=(0, 255, 0))

# Display
display_multiple_images(
    [img, img_orb],
    ['Original Image', f'ORB Keypoints ({len(keypoints_orb)} detected)'],
    rows=1, cols=2, figsize=(15, 5)
)

print(f'\n✅ ORB 檢測結果:')
print('='*60)
print(f'關鍵點數量: {len(keypoints_orb)}')
print(f'描述子維度: {descriptors_orb.shape}')
print(f'描述子類型: {descriptors_orb.dtype} (二進位)')
print(f'每個描述子: 256 位元 (32 bytes)')

if len(keypoints_orb) > 0:
    print(f'\n第一個描述子 (前16個位元組):')
    print(descriptors_orb[0][:16])

### 4.2 ORB 旋轉不變性驗證

In [ ]:
# Test ORB rotation invariance
angles = [0, 45, 90, 135]
results = []
titles = []

for angle in angles:
    # Rotate image
    h, w = gray.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(gray, M, (w, h))
    
    # Detect keypoints
    kp, desc = orb.detectAndCompute(rotated, None)
    
    # Draw keypoints
    img_kp = cv2.cvtColor(rotated, cv2.COLOR_GRAY2BGR)
    img_kp = cv2.drawKeypoints(img_kp, kp, None, color=(0, 255, 0))
    
    results.append(img_kp)
    titles.append(f'Rotation {angle}°\n{len(kp)} keypoints')

display_multiple_images(results, titles, rows=1, cols=4, figsize=(20, 5))

print('\n💡 觀察: ORB 在不同旋轉角度下都能檢測到穩定的特徵點')

## 5. BRIEF (Binary Robust Independent Elementary Features)

### 5.1 BRIEF 原理

**BRIEF** 是一種二進位描述子，不提供關鍵點檢測，只提供特徵描述。

#### 核心思想

```
BRIEF 描述子生成:

1. 在關鍵點周圍選擇像素對
   └─> 通常選擇 256 對像素

2. 比較每對像素的亮度
   └─> If I(p1) < I(p2): bit = 1
   └─> Else: bit = 0

3. 生成 256 位元二進位字串
   └─> 非常緊湊的描述子
```

#### BRIEF 特點

- ✅ **極快**: 只需簡單的像素比較
- ✅ **記憶體小**: 256 位元 = 32 bytes
- ✅ **匹配快**: 使用漢明距離（XOR 操作）
- ❌ **無旋轉不變性**: 影像旋轉會失效
- ❌ **無尺度不變性**: 需要其他檢測器提供關鍵點

### ⚠️ 重要提示

BRIEF 只是描述子，需要搭配其他特徵檢測器（如 FAST）使用。

In [ ]:
# BRIEF requires a separate keypoint detector
try:
    # Use FAST for keypoint detection
    fast = cv2.FastFeatureDetector_create()
    keypoints_fast = fast.detect(gray, None)
    
    # Use BRIEF for description
    brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
    keypoints_brief, descriptors_brief = brief.compute(gray, keypoints_fast)
    
    # Draw keypoints
    img_brief = cv2.drawKeypoints(img, keypoints_brief, None, color=(255, 0, 0))
    
    display_multiple_images(
        [img, img_brief],
        ['Original', f'FAST + BRIEF ({len(keypoints_brief)} keypoints)'],
        rows=1, cols=2, figsize=(15, 5)
    )
    
    print(f'✅ BRIEF 檢測結果:')
    print(f'關鍵點數量 (FAST): {len(keypoints_fast)}')
    print(f'描述子數量: {len(keypoints_brief)}')
    print(f'描述子維度: {descriptors_brief.shape}')
    print(f'描述子類型: {descriptors_brief.dtype}')
    
except AttributeError:
    print('⚠️ BRIEF 在此配置中不可用')
    print('   請使用 ORB（包含改進的 rBRIEF）')

## 6. BRISK (Binary Robust Invariant Scalable Keypoints)

### 6.1 BRISK 原理

**BRISK** 是另一個開源的二進位特徵檢測器，結合了 FAST 和 BRIEF 的優點。

#### 核心特性

```
BRISK 改進:

1. 尺度空間關鍵點檢測
   └─> 在不同尺度上使用 FAST

2. 方向估計
   └─> 使用長距離像素對估計方向

3. 採樣模式
   └─> 使用同心圓採樣模式

4. 512 位元描述子
   └─> 比 ORB 的 256 位元更具區分性
```

#### BRISK 優勢

- ✅ **尺度不變**: 支援多尺度檢測
- ✅ **旋轉不變**: 有方向估計
- ✅ **速度快**: 接近 ORB 的速度
- ✅ **開源**: 無專利限制
- ⭐ **平衡性好**: 速度與準確度的良好平衡

In [ ]:
# Create BRISK detector
brisk = cv2.BRISK_create()

# Detect keypoints and compute descriptors
keypoints_brisk, descriptors_brisk = brisk.detectAndCompute(gray, None)

# Draw keypoints
img_brisk = cv2.drawKeypoints(img, keypoints_brisk, None, 
                               flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Display
display_multiple_images(
    [img, img_brisk],
    ['Original Image', f'BRISK Keypoints ({len(keypoints_brisk)} detected)'],
    rows=1, cols=2, figsize=(15, 5)
)

print(f'\n✅ BRISK 檢測結果:')
print('='*60)
print(f'關鍵點數量: {len(keypoints_brisk)}')
print(f'描述子維度: {descriptors_brisk.shape}')
print(f'描述子類型: {descriptors_brisk.dtype} (二進位)')
print(f'每個描述子: 512 位元 (64 bytes)')

print(f'\n💡 BRISK vs ORB:')
print(f'   - BRISK 描述子更長 (512 vs 256 位元)')
print(f'   - BRISK 通常檢測更多關鍵點')
print(f'   - BRISK 有更好的尺度不變性')

## 7. 特徵檢測器比較

### 7.1 同時比較所有檢測器

In [ ]:
# Compare all detectors
detectors = {
    'SIFT': sift,
    'ORB': orb,
    'BRISK': brisk
}

results = []
titles = []
stats = []

for name, detector in detectors.items():
    # Detect keypoints
    kp, desc = detector.detectAndCompute(gray, None)
    
    # Draw keypoints
    img_kp = cv2.drawKeypoints(img, kp, None, color=(0, 255, 0))
    
    results.append(img_kp)
    titles.append(f'{name}\n{len(kp)} keypoints')
    
    stats.append({
        'name': name,
        'keypoints': len(kp),
        'descriptor_size': desc.shape[1] if desc is not None else 0,
        'descriptor_type': desc.dtype if desc is not None else 'None'
    })

# Display comparison
display_multiple_images(results, titles, rows=1, cols=3, figsize=(18, 6))

# Print statistics
print('\n特徵檢測器統計比較:')
print('='*80)
print(f'{'檢測器':10} | {'關鍵點數':>10} | {'描述子維度':>12} | {'類型':>10}')
print('-'*80)
for s in stats:
    print(f"{s['name']:10} | {s['keypoints']:>10} | {s['descriptor_size']:>12} | {str(s['descriptor_type']):>10}")

### 7.2 性能基準測試

In [ ]:
# Performance benchmark
test_img = cv2.resize(gray, (640, 480))

print('\n效能測試 (640x480 影像):')  
print('='*80)
print(f'{'檢測器':10} | {'平均時間 (ms)':>15} | {'FPS':>10} | {'關鍵點數':>10}')
print('-'*80)

for name, detector in detectors.items():
    # Benchmark
    def detect():
        return detector.detectAndCompute(test_img, None)
    
    stats = benchmark_function(detect, iterations=10)
    kp, _ = detect()
    
    avg_time = stats['average'] * 1000
    fps = 1.0 / stats['average'] if stats['average'] > 0 else 0
    
    print(f'{name:10} | {avg_time:13.2f} ms | {fps:8.1f} | {len(kp):>10}')

print('\n💡 觀察:')
print('   - ORB 最快，適合實時應用')
print('   - BRISK 速度與品質的良好平衡')
print('   - SIFT 最穩定但速度較慢')

## 8. 特徵匹配 (Feature Matching)

### 8.1 BFMatcher (Brute-Force Matcher)

**BFMatcher** 是最簡單的特徵匹配器，對每個描述子進行暴力搜索。

#### 工作原理

```
BFMatcher 匹配流程:

1. 取第一張影像的第一個描述子
2. 計算與第二張影像所有描述子的距離
3. 找到距離最小的作為匹配
4. 重複對所有描述子進行匹配

距離度量:
- L2/L1 距離: 用於 SIFT/SURF (浮點描述子)
- Hamming 距離: 用於 ORB/BRISK (二進位描述子)
```

#### 距離計算

**Hamming 距離**: 兩個二進位字串中不同位元的數量
```
例: 
A = 11010101
B = 10110001
Hamming(A,B) = 3 (有3個位元不同)
```

In [ ]:
# Load two images for matching
img1_path = '../assets/images/basic/opencv.jpg'
img2_path = '../assets/images/basic/opencv.jpg'  # Same image, slightly transformed

if os.path.exists(img1_path):
    img1 = cv2.imread(img1_path)
    img1 = cv2.resize(img1, (400, 300))
else:
    img1 = np.ones((300, 400, 3), dtype=np.uint8) * 255
    cv2.rectangle(img1, (50, 50), (350, 250), (0, 0, 0), -1)
    cv2.circle(img1, (200, 150), 60, (255, 255, 255), -1)

# Create transformed version of img1
h, w = img1.shape[:2]
M = cv2.getRotationMatrix2D((w//2, h//2), 15, 0.9)  # Rotate 15° and scale 0.9x
img2 = cv2.warpAffine(img1, M, (w, h))

gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Detect keypoints and descriptors using ORB
orb_match = cv2.ORB_create(nfeatures=500)
kp1, des1 = orb_match.detectAndCompute(gray1, None)
kp2, des2 = orb_match.detectAndCompute(gray2, None)

# Create BFMatcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors
matches = bf.match(des1, des2)

# Sort matches by distance (best matches first)
matches = sorted(matches, key=lambda x: x.distance)

# Draw first 50 matches
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None,
                               flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display
plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
plt.title(f'BFMatcher: Top 50 matches (out of {len(matches)} total)', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

print(f'\n✅ BFMatcher 匹配結果:')
print('='*60)
print(f'影像1 關鍵點: {len(kp1)}')
print(f'影像2 關鍵點: {len(kp2)}')
print(f'總匹配數: {len(matches)}')
print(f'\n前5個最佳匹配的距離:')
for i, m in enumerate(matches[:5]):
    print(f'  Match {i+1}: distance = {m.distance:.1f}')

### 8.2 FLANN Matcher (Fast Library for Approximate Nearest Neighbors)

**FLANN** 是一個快速近似最近鄰搜索庫，比 BFMatcher 更快。

#### 核心優勢

```
FLANN vs BFMatcher:

BFMatcher:
- 精確匹配
- 簡單直接
- 適合小數據集

FLANN:
- 近似匹配（但非常接近精確解）
- 使用優化的索引結構 (KD-Tree, LSH)
- 適合大數據集
- 速度快 10-100 倍
```

#### FLANN 參數

- **KD-Tree**: 用於浮點描述子 (SIFT, SURF)
- **LSH**: 用於二進位描述子 (ORB, BRISK)

In [ ]:
# Detect features using SIFT (for FLANN with KD-Tree)
kp1_sift, des1_sift = sift.detectAndCompute(gray1, None)
kp2_sift, des2_sift = sift.detectAndCompute(gray2, None)

# FLANN parameters for SIFT
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)  # Higher = more accurate but slower

# Create FLANN matcher
flann = cv2.FlannBasedMatcher(index_params, search_params)

# Match descriptors (k=2 for ratio test)
matches_flann = flann.knnMatch(des1_sift, des2_sift, k=2)

# Apply Lowe's ratio test
good_matches = []
for m, n in matches_flann:
    if m.distance < 0.7 * n.distance:  # Ratio threshold
        good_matches.append(m)

# Draw matches
img_flann = cv2.drawMatches(img1, kp1_sift, img2, kp2_sift, good_matches[:50], None,
                             flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display
plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(img_flann, cv2.COLOR_BGR2RGB))
plt.title(f'FLANN Matcher: Top 50 good matches (out of {len(good_matches)} total)', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

print(f'\n✅ FLANN 匹配結果:')
print('='*60)
print(f'影像1 關鍵點: {len(kp1_sift)}')
print(f'影像2 關鍵點: {len(kp2_sift)}')
print(f'原始匹配數: {len(matches_flann)}')
print(f'通過 Ratio Test 的匹配數: {len(good_matches)}')
print(f'匹配率: {len(good_matches)/len(matches_flann)*100:.1f}%')

print(f'\n💡 Lowe\'s Ratio Test:')
print(f'   - 比較最佳匹配和次佳匹配的距離')
print(f'   - 如果比例 < 0.7，認為是好的匹配')
print(f'   - 可以過濾掉模糊的匹配')

### 8.3 FLANN with ORB (LSH Index)

In [ ]:
# FLANN parameters for binary descriptors (ORB)
FLANN_INDEX_LSH = 6
index_params_lsh = dict(algorithm=FLANN_INDEX_LSH,
                        table_number=6,      # 12
                        key_size=12,         # 20
                        multi_probe_level=1) # 2
search_params_lsh = dict(checks=50)

# Create FLANN matcher for binary descriptors
flann_orb = cv2.FlannBasedMatcher(index_params_lsh, search_params_lsh)

# Match ORB descriptors
matches_orb_flann = flann_orb.knnMatch(des1, des2, k=2)

# Apply ratio test
good_matches_orb = []
for match_pair in matches_orb_flann:
    if len(match_pair) == 2:  # Ensure we have 2 matches
        m, n = match_pair
        if m.distance < 0.75 * n.distance:
            good_matches_orb.append(m)

# Draw matches
img_orb_flann = cv2.drawMatches(img1, kp1, img2, kp2, good_matches_orb[:50], None,
                                 flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display
plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(img_orb_flann, cv2.COLOR_BGR2RGB))
plt.title(f'FLANN (LSH) + ORB: Top 50 matches (out of {len(good_matches_orb)} total)', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

print(f'\n✅ FLANN (LSH) + ORB 匹配結果:')
print('='*60)
print(f'通過 Ratio Test 的匹配數: {len(good_matches_orb)}')
print(f'\n💡 LSH (Locality Sensitive Hashing):')
print(f'   - 專為二進位描述子設計')
print(f'   - 使用哈希表加速搜索')
print(f'   - 比 KD-Tree 更適合高維二進位資料')

## 9. Homography 變換與影像配準

### 9.1 Homography 原理

**Homography** 是一種將一個平面投影到另一個平面的變換。

#### 數學定義

Homography 矩陣 H (3x3) 描述兩個平面間的透視變換:

```
[x']   [h11  h12  h13]   [x]
[y'] = [h21  h22  h23] × [y]
[1 ]   [h31  h32  h33]   [1]
```

#### 應用場景

```
Homography 應用:

1. 影像拼接 (Image Stitching)
   └─> 全景圖製作

2. 視角校正 (Perspective Correction)
   └─> 文檔掃描、道路標線

3. 物體識別 (Object Recognition)
   └─> 平面物體檢測與追蹤

4. 擴增實境 (AR)
   └─> 虛擬物體疊加
```

#### RANSAC 演算法

RANSAC (Random Sample Consensus) 用於從含有外點 (outliers) 的匹配中估計 Homography:

```
RANSAC 流程:

1. 隨機選擇 4 對匹配點
2. 計算 Homography 矩陣
3. 計算所有點的誤差
4. 統計內點 (inliers) 數量
5. 重複多次，選擇內點最多的模型
```

In [ ]:
# Need at least 4 matches for Homography
MIN_MATCH_COUNT = 10

if len(good_matches) >= MIN_MATCH_COUNT:
    # Extract location of good matches
    src_pts = np.float32([kp1_sift[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2_sift[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    
    # Find Homography matrix using RANSAC
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask.ravel().tolist()
    
    # Draw bounding box in img2
    h, w = gray1.shape
    pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, M)
    
    img2_box = img2.copy()
    img2_box = cv2.polylines(img2_box, [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)
    
    # Draw only inliers
    draw_params = dict(matchColor=(0, 255, 0),
                       singlePointColor=None,
                       matchesMask=matchesMask,
                       flags=2)
    
    img_homography = cv2.drawMatches(img1, kp1_sift, img2_box, kp2_sift, 
                                      good_matches, None, **draw_params)
    
    # Display
    plt.figure(figsize=(18, 9))
    plt.imshow(cv2.cvtColor(img_homography, cv2.COLOR_BGR2RGB))
    plt.title(f'Homography: {sum(matchesMask)} inliers out of {len(good_matches)} matches', 
              fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print(f'\n✅ Homography 計算結果:')
    print('='*60)
    print(f'總匹配數: {len(good_matches)}')
    print(f'內點 (inliers): {sum(matchesMask)}')
    print(f'外點 (outliers): {len(matchesMask) - sum(matchesMask)}')
    print(f'內點比例: {sum(matchesMask)/len(matchesMask)*100:.1f}%')
    
    print(f'\nHomography 矩陣:')
    print(M)
    
else:
    print(f'⚠️ 匹配點不足 - 需要至少 {MIN_MATCH_COUNT} 個匹配點')
    print(f'   當前僅有 {len(good_matches)} 個匹配點')

### 9.2 應用 Homography 進行影像對齊

In [ ]:
if len(good_matches) >= MIN_MATCH_COUNT and M is not None:
    # Warp img1 to align with img2
    h, w = img2.shape[:2]
    img1_aligned = cv2.warpPerspective(img1, M, (w, h))
    
    # Create difference image
    diff = cv2.absdiff(img2, img1_aligned)
    diff_gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    # Display
    display_multiple_images(
        [img1, img2, img1_aligned, diff_gray],
        ['Original Image 1', 'Image 2 (Rotated)', 'Image 1 Aligned', 'Difference'],
        rows=2, cols=2, figsize=(14, 10)
    )
    
    # Calculate alignment quality
    mse = np.mean((img2.astype(float) - img1_aligned.astype(float))**2)
    psnr = 20 * np.log10(255.0 / np.sqrt(mse)) if mse > 0 else float('inf')
    
    print(f'\n✅ 影像對齊品質:')
    print('='*60)
    print(f'MSE:  {mse:.2f}')
    print(f'PSNR: {psnr:.2f} dB')
    print(f'\n💡 PSNR 越高表示對齊越好（通常 > 30dB 為良好對齊）')
else:
    print('⚠️ 無法計算 Homography，匹配點不足')

## 10. 特徵檢測器完整比較表

### 10.1 技術特性比較

| 特徵 | SIFT | SURF | ORB | BRISK | BRIEF |
|------|------|------|-----|-------|-------|
| **尺度不變** | ✅ | ✅ | ⭐ | ✅ | ❌ |
| **旋轉不變** | ✅ | ✅ | ✅ | ✅ | ❌ |
| **仿射不變** | ⭐ | ⭐ | ❌ | ❌ | ❌ |
| **光照穩定** | ✅ | ✅ | ⭐ | ⭐ | ⭐ |
| **速度** | ⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| **魯棒性** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ |
| **記憶體** | 128維/float | 64維/float | 256位元 | 512位元 | 256位元 |
| **專利** | 已過期 | 有專利⚠️ | 開源✅ | 開源✅ | 開源✅ |

### 10.2 應用場景推薦

| 場景 | 推薦檢測器 | 理由 |
|------|-----------|------|
| **實時應用** | ORB, BRISK | 速度最快，適合移動設備 |
| **高精度匹配** | SIFT | 最穩定，準確度最高 |
| **影像拼接** | SIFT, SURF | 需要尺度和旋轉不變性 |
| **物體追蹤** | ORB | 速度快，實時性能好 |
| **AR 應用** | BRISK, ORB | 平衡速度與準確度 |
| **嵌入式系統** | ORB | 記憶體佔用小，計算快 |
| **商業應用** | ORB, BRISK | 無專利限制 |

### 10.3 匹配器選擇指南

| 描述子類型 | 推薦匹配器 | 距離度量 | 參數設置 |
|-----------|----------|---------|----------|
| SIFT, SURF | FLANN (KD-Tree) | L2 範數 | trees=5, checks=50 |
| ORB, BRISK | BFMatcher | Hamming | crossCheck=True |
| ORB, BRISK | FLANN (LSH) | Hamming | table_number=6, key_size=12 |

### 10.4 性能對比總結

```
速度排名:   ORB > BRISK > SURF > SIFT
準確度:     SIFT > SURF > BRISK > ORB
魯棒性:     SIFT > SURF ≈ BRISK > ORB
記憶體:     BRIEF ≈ ORB < SURF < BRISK < SIFT

綜合推薦:
- 研究/高精度: SIFT
- 產品/實時: ORB
- 平衡方案: BRISK
```

## 11. 實戰練習

### 練習 1: 實現簡單的物體檢測

In [ ]:
# TODO: 使用特徵匹配在場景中檢測目標物體

def detect_object(template, scene, detector='ORB', min_matches=10):
    """
    Detect template object in scene using feature matching
    
    Args:
        template: Template image (object to find)
        scene: Scene image (where to search)
        detector: 'SIFT', 'ORB', or 'BRISK'
        min_matches: Minimum number of matches required
    
    Returns:
        detected_image: Scene with bounding box
        num_matches: Number of good matches found
    """
    # Convert to grayscale
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY) if len(template.shape) == 3 else template
    scene_gray = cv2.cvtColor(scene, cv2.COLOR_BGR2GRAY) if len(scene.shape) == 3 else scene
    
    # Create detector
    if detector == 'SIFT':
        det = sift
        matcher = cv2.FlannBasedMatcher(
            dict(algorithm=1, trees=5),
            dict(checks=50)
        )
    elif detector == 'ORB':
        det = cv2.ORB_create(nfeatures=1000)
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    else:  # BRISK
        det = cv2.BRISK_create()
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    
    # Detect and compute
    kp1, des1 = det.detectAndCompute(template_gray, None)
    kp2, des2 = det.detectAndCompute(scene_gray, None)
    
    if des1 is None or des2 is None:
        return scene.copy(), 0
    
    # Match
    matches = matcher.knnMatch(des1, des2, k=2)
    
    # Apply ratio test
    good = []
    for match_pair in matches:
        if len(match_pair) == 2:
            m, n = match_pair
            if m.distance < 0.75 * n.distance:
                good.append(m)
    
    result = scene.copy()
    
    # Draw bounding box if enough matches
    if len(good) >= min_matches:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        if M is not None:
            h, w = template_gray.shape
            pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
            dst = cv2.perspectiveTransform(pts, M)
            
            result = cv2.polylines(result, [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)
            
            # Add text
            cv2.putText(result, f'{detector}: {len(good)} matches', (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    return result, len(good)

# Test object detection
template = img1[50:250, 100:300]  # Extract template from img1
scene = img2

# Try different detectors
results = []
titles = []

for det_name in ['SIFT', 'ORB', 'BRISK']:
    detected, n_matches = detect_object(template, scene, detector=det_name)
    results.append(detected)
    titles.append(f'{det_name}\n{n_matches} matches')

# Add template
results.insert(0, cv2.cvtColor(template, cv2.COLOR_BGR2RGB))
titles.insert(0, 'Template')

display_multiple_images(results, titles, rows=1, cols=4, figsize=(20, 5))

print('✅ 物體檢測完成')
print('💡 觀察不同檢測器在物體檢測任務上的表現差異')

### 練習 2: 比較不同檢測器的旋轉魯棒性

In [ ]:
# TODO: 測試不同旋轉角度下的匹配穩定性

angles = [0, 30, 60, 90, 120, 150, 180]
detectors_test = {'SIFT': sift, 'ORB': orb, 'BRISK': brisk}

results_rotation = {name: [] for name in detectors_test.keys()}

for angle in angles:
    # Rotate image
    h, w = gray.shape
    M_rot = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
    rotated = cv2.warpAffine(gray, M_rot, (w, h))
    
    for name, det in detectors_test.items():
        # Detect and match
        kp1, des1 = det.detectAndCompute(gray, None)
        kp2, des2 = det.detectAndCompute(rotated, None)
        
        if des1 is not None and des2 is not None:
            if name == 'SIFT':
                bf_temp = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
            else:
                bf_temp = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
            
            matches_temp = bf_temp.match(des1, des2)
            results_rotation[name].append(len(matches_temp))
        else:
            results_rotation[name].append(0)

# Plot results
plt.figure(figsize=(12, 6))
for name, counts in results_rotation.items():
    plt.plot(angles, counts, marker='o', label=name, linewidth=2)

plt.xlabel('Rotation Angle (degrees)', fontsize=12)
plt.ylabel('Number of Matches', fontsize=12)
plt.title('Rotation Robustness Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('\n旋轉魯棒性測試結果:')
print('='*60)
for name, counts in results_rotation.items():
    avg_matches = np.mean(counts)
    std_matches = np.std(counts)
    print(f'{name:10} | 平均匹配數: {avg_matches:6.1f} | 標準差: {std_matches:5.1f}')

print('\n💡 標準差越小表示旋轉魯棒性越好')

## 12. 總結

### 重點回顧

1. **特徵檢測器家族**
   - SIFT: 最穩定，但速度慢
   - SURF: SIFT 加速版，有專利限制
   - ORB: 開源，速度最快，適合實時
   - BRISK: 平衡速度與準確度
   - BRIEF: 只是描述子，需搭配其他檢測器

2. **特徵匹配方法**
   - BFMatcher: 精確但慢，適合小數據集
   - FLANN: 快速近似匹配，適合大數據集
   - Ratio Test: Lowe's 方法過濾不良匹配

3. **Homography 應用**
   - 影像拼接
   - 視角校正
   - 物體檢測
   - RANSAC 過濾外點

4. **實際應用建議**
   - 研究/高精度 → SIFT
   - 商業/實時 → ORB
   - 平衡方案 → BRISK

### 決策樹

```
選擇特徵檢測器?
├─ 需要實時處理?
│   ├─ 是 → ORB (最快)
│   └─ 否 → 繼續
│
├─ 需要最高準確度?
│   ├─ 是 → SIFT (最穩定)
│   └─ 否 → 繼續
│
├─ 商業應用（避免專利）?
│   ├─ 是 → ORB 或 BRISK
│   └─ 否 → SURF (如果可用)
│
└─ 需要平衡方案?
    └─ BRISK (速度與準確度平衡)
```

### 下一步學習

- [x] 特徵描述子 ✅ ← **你在這裡**
- [ ] 光流追蹤 (Optical Flow)
- [ ] 物體追蹤演算法
- [ ] 深度學習特徵 (CNN Features)
- [ ] 影像拼接專案

### 參考資源

- OpenCV Feature Detection: https://docs.opencv.org/4.x/db/d27/tutorial_py_table_of_contents_feature2d.html
- SIFT Paper: Lowe, D.G. (2004)
- ORB Paper: Rublee et al. (2011)
- 本專案 utils 模組: `utils/image_utils.py`, `utils/performance.py`